In [153]:
import numpy as np

In [154]:
#   .txt
file_name = '01 - The Fellowship Of The Ring.txt'

### Deliverable 1


In [155]:
file = open(file_name, 'r')
content = file.read()
file.close()

num_char = len(content)

unique_char = set(content)
num_unique_char = len(unique_char)

# each character is encoded by the same number of bits
average_bit = np.round((np.log(num_unique_char)), 2)
total_bit = average_bit * num_char

# print(content, end='\n\n')
# print(unique_char, end='\n\n')

print('Total number of characters: {} \nNumber of distinct characters: {} \nTotal number of bits needed to encode the text: {} \nAverage number of bits per character: {}'.format(
    num_char, num_unique_char, total_bit, average_bit))

Total number of characters: 1024530 
Number of distinct characters: 81 
Total number of bits needed to encode the text: 4497686.699999999 
Average number of bits per character: 4.39


### Deliverable 2


In [156]:
file = open(file_name, 'r')
content = file.read()
file.close()

num_char = len(content)

char_freq = dict()
for char in content:
    char_freq[char] = char_freq.get(char, 0) + 1

# sort unique characters by frequency in descending order
sorted_unique_chars = sorted(
    char_freq.keys(), key=lambda char: char_freq[char], reverse=True)

print(sorted_unique_chars)

total_bit = 0
i = 1
for char in sorted_unique_chars:
    total_bit += char_freq[char] * i
    i += 1

average_bit = np.round(total_bit / num_char, 2)

print('Total numer of bits: {} \nAverage number of bits: {}'.format(
    total_bit, average_bit))

[' ', 'e', 't', 'a', 'o', 'n', 'h', 'i', 'r', 's', 'd', 'l', 'u', 'w', 'g', 'f', 'm', 'y', 'c', ',', '.', 'b', 'p', "'", 'k', 'v', '\n', 'I', 'T', 'B', 'S', 'F', 'A', 'H', '-', '!', 'G', ';', 'W', 'M', '`', 'E', 'R', '_', '?', ':', 'N', 'L', 'O', 'D', '�', 'C', 'Y', 'q', 'P', 'x', 'j', '"', 'z', '(', ')', 'U', 'K', '1', 'V', '=', 'J', '2', '3', '4', '9', 'Q', '0', '7', '6', '8', '5', '\t', '/', 'X', 'Z']
Total numer of bits: 8674542 
Average number of bits: 8.47


### Deliverable 3


In [157]:
class Node:
    def __init__(self, chars, freq, left=None, right=None):
        self.chars = chars
        self.freq = freq
        self.left = left
        self.right = right

In [158]:
def get_char_freq(file_name):
    file = open(file_name, 'r')
    content = file.read()
    file.close()

    char_freq = dict()
    for char in content:
        char_freq[char] = char_freq.get(char, 0) + 1

    return char_freq

In [159]:
def create_node(char_freq):
    nodes = list()

    for char, freq in char_freq.items():
        nodes.append(Node([char], freq))

    return nodes

In [160]:
def merge_trees(left_root, right_root):
    chars = list(set(left_root.chars + right_root.chars))
    freq = left_root.freq + right_root.freq

    return Node(chars, freq, left_root, right_root)

In [161]:
def create_binary_tree(nodes):
    sorted_nodes = sorted(nodes, key=lambda x: x.freq)

    root = sorted_nodes[0]
    for i in range(1, len(sorted_nodes), 1):
        root = merge_trees(sorted_nodes[i], root)

    return root

In [162]:
# HELPER
def visualise_tree(root, level=0):
    print(('\t' * level) + '{} - {} [{}]'.format(level, root.chars, root.freq))

    level += 1
    if root.left:
        visualise_tree(root.left, level)

    if root.right:
        visualise_tree(root.right, level)

In [163]:
test_case = {'a': 20, 'b': 2, 'c': 1}
root_test_case = create_binary_tree(create_node(test_case))

visualise_tree(root_test_case)

0 - ['a', 'b', 'c'] [23]
	1 - ['a'] [20]
	1 - ['b', 'c'] [3]
		2 - ['b'] [2]
		2 - ['c'] [1]


### Deliverable 4


In [164]:
def find_in_tree(char, root):
    level = 0
    while root:
        if not root.left:
            if char == root.chars[0]:
                return level
            else:
                break

        level += 1
        if char == root.left.chars[0]:
            return level

        root = root.right

    return 'None'

In [165]:
find_in_tree('a', root_test_case)

1

In [166]:
find_in_tree('b', root_test_case)

2

In [167]:
find_in_tree('c', root_test_case)

2

In [168]:
find_in_tree('d', root_test_case)

'None'

### Deliverable 5


In [197]:
def tree_from_file(file_name):
    file = open(file_name, 'r')
    content = file.read()
    file.close()

    num_char = len(content)

    char_freq = dict()
    for char in content:
        char_freq[char] = char_freq.get(char, 0) + 1

    # sort unique characters by frequency in descending order
    sorted_unique_chars = sorted(
        char_freq.keys(), key=lambda char: char_freq[char], reverse=True)

    def create_binary_tree(sorted_unique_chars):
        nonlocal char_freq

        freq = sum(char_freq[char] for char in sorted_unique_chars)
        node = Node(sorted_unique_chars, freq)

        n = len(sorted_unique_chars)
        if n == 1:
            return node
        elif n == 2:
            node.left = Node([sorted_unique_chars[0]],
                             char_freq[sorted_unique_chars[0]])
            node.right = Node([sorted_unique_chars[1]],
                              char_freq[sorted_unique_chars[1]])
            return node

        left = 0
        right = freq

        for i in range(0, n, 1):
            if left >= right:
                node.left = create_binary_tree(sorted_unique_chars[0:i])
                node.right = create_binary_tree(sorted_unique_chars[i:])
                return node

            left += char_freq[sorted_unique_chars[i]]
            right -= char_freq[sorted_unique_chars[i]]

    node = create_binary_tree(sorted_unique_chars)

    def calculate_bit(node, level=0):
        if node.left is None and node.right is None:
            return node.freq * level
        else:
            level += 1
            return calculate_bit(node.left, level) + calculate_bit(node.right, level)

    total_bit = calculate_bit(node)

    average_bit = np.round(total_bit / num_char, 2)

    print('Total number of characters: {} \nNumber of distinct characters: {} \nTotal number of bits needed to encode the text: {} \nAverage number of bits per character: {}'.format(
        num_char, len(sorted_unique_chars), total_bit, average_bit))

    visualise_tree(node)

In [196]:
tree_from_file(file_name)

Total number of characters: 1024530 
Number of distinct characters: 81 
Total number of bits needed to encode the text: 4581141 
Average number of bits per character: 4.47
